# 📊 Análisis Exploratorio de Datos (EDA) - Marketing Mix Modeling

Este notebook presenta un análisis exploratorio limpio de los datos de marketing para el proyecto de Marketing Mix Modeling (MMM).

**Objetivo:** Entender la estructura de los datos, identificar patrones y relaciones entre variables de inversión publicitaria y ventas.

**Secciones:**
1. Importación de librerías y configuración
2. Carga de datos
3. Inspección inicial y estadísticas descriptivas
4. Análisis de series temporales
5. Análisis de correlación
6. Distribuciones y outliers
7. Conclusiones para el modelado

## 1. Importación de librerías y configuración

## 2. Carga de datos

## 3. Inspección inicial y estadísticas descriptivas

In [ ]:
# Verificar valores nulos
print("🔍 Valores nulos por columna:\n")
null_counts = df.isnull().sum()
if null_counts.sum() == 0:
    print("✅ No hay valores nulos en el dataset")
else:
    print(null_counts[null_counts > 0])

In [ ]:
# Crear gráficos de series temporales
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Ventas', 'TV', 'Radio', 'Newspaper'),
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

# Identificar columnas (buscar patrones comunes)
sales_col = [c for c in df.columns if 'Sales' in c or 'Ventas' in c][0] if any('Sales' in c or 'Ventas' in c for c in df.columns) else df.columns[-1]
tv_col = [c for c in df.columns if 'TV' in c][0] if any('TV' in c for c in df.columns) else None
radio_col = [c for c in df.columns if 'Radio' in c][0] if any('Radio' in c for c in df.columns) else None
news_col = [c for c in df.columns if 'Newspaper' in c or 'News' in c][0] if any('Newspaper' in c or 'News' in c for c in df.columns) else None

# Ventas
fig.add_trace(
    go.Scatter(x=df.index, y=df[sales_col], mode='lines', name='Sales', line=dict(color='blue')),
    row=1, col=1
)

# TV (si existe)
if tv_col:
    fig.add_trace(
        go.Scatter(x=df.index, y=df[tv_col], mode='lines', name='TV', line=dict(color='orange')),
        row=1, col=2
    )

# Radio (si existe)
if radio_col:
    fig.add_trace(
        go.Scatter(x=df.index, y=df[radio_col], mode='lines', name='Radio', line=dict(color='green')),
        row=2, col=1
    )

# Newspaper (si existe)
if news_col:
    fig.add_trace(
        go.Scatter(x=df.index, y=df[news_col], mode='lines', name='Newspaper', line=dict(color='red')),
        row=2, col=2
    )

fig.update_layout(height=600, title_text="Evolución temporal de variables", showlegend=False)
fig.show()

In [ ]:
# Matriz de correlación
correlation_matrix = df.corr(numeric_only=True)

# Visualizar con heatmap
fig = go.Figure(data=go.Heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns,
    y=correlation_matrix.index,
    colorscale='RdBu',
    zmid=0,
    text=correlation_matrix.values,
    texttemplate='%{text:.2f}',
    textfont={"size": 10},
    colorbar=dict(title="Correlación")
))

fig.update_layout(
    title="Matriz de Correlación",
    xaxis_title="Variables",
    yaxis_title="Variables",
    height=500,
    width=700
)

fig.show()

# Mostrar correlaciones con Sales
print("\n📊 Correlación con Ventas (ordenado):\n")
sales_corr = correlation_matrix[sales_col].sort_values(ascending=False)
print(sales_corr)

In [ ]:
# Boxplots para detectar outliers
fig = go.Figure()

for col in numeric_cols[:4]:
    fig.add_trace(go.Box(y=df[col], name=col))

fig.update_layout(
    title="Boxplots - Detección de outliers",
    yaxis_title="Valor",
    height=500
)

fig.show()

## 7. Conclusiones para el modelado

### Observaciones clave:

1. **Calidad de los datos**: 
   - ✅ Sin valores nulos
   - ✅ Tipos de datos correctos
   - ⚠️ Revisar outliers si existen

2. **Correlaciones**:
   - Identificar qué canales tienen mayor correlación con ventas
   - Considerar multicolinealidad entre variables de medios

3. **Transformaciones necesarias**:
   - **Adstock**: Capturar efectos de arrastre temporal
   - **Saturación (Hill)**: Modelar rendimientos decrecientes
   - **Estandarización**: Mejorar convergencia del modelo bayesiano

4. **Próximos pasos**:
   - Aplicar transformaciones usando `mmm_core.transforms`
   - Construir modelo bayesiano con PyMC
   - Evaluar contribuciones y ROI/ROAS por canal

### Recomendaciones:

- Usar **ADVI** para ajuste rápido del modelo (desarrollo)
- Considerar **NUTS** para análisis final (más preciso)
- Validar supuestos del modelo con diagnósticos bayesianos

In [ ]:
# Seleccionar columnas numéricas (excepto índice/fecha)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Crear histogramas
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=numeric_cols[:4]
)

for idx, col in enumerate(numeric_cols[:4]):
    row = (idx // 2) + 1
    col_num = (idx % 2) + 1
    
    fig.add_trace(
        go.Histogram(x=df[col], name=col, nbinsx=30),
        row=row, col=col_num
    )

fig.update_layout(height=600, title_text="Distribución de variables", showlegend=False)
fig.show()

## 6. Distribuciones y outliers

Examinamos la distribución de cada variable para detectar posibles outliers o patrones inusuales.

## 5. Análisis de correlación

Analizamos las relaciones entre variables para identificar cuáles tienen mayor impacto en las ventas.

## 4. Análisis de series temporales

Visualizamos la evolución de las ventas y la inversión en medios a lo largo del tiempo.

In [ ]:
# Estadísticas descriptivas
print("📈 Estadísticas descriptivas:\n")
df.describe()

In [ ]:
# Información del dataset
print("ℹ️ Información del dataset:\n")
df.info()

In [ ]:
# Vista previa de los datos
print("📊 Primeras filas del dataset:\n")
df.head(10)

In [ ]:
# Cargar datos desde la carpeta data/
data_path = "../data/Basemediosfinal.csv"

try:
    df_raw = data.load_base_data(data_path)
    print(f"✅ Datos cargados: {df_raw.shape[0]} filas, {df_raw.shape[1]} columnas")
except FileNotFoundError:
    print("⚠️ Archivo no encontrado. Generando datos sintéticos...")
    df_raw = data.generate_synthetic_data(n=200, seed=42)
    print(f"✅ Datos sintéticos generados: {df_raw.shape[0]} filas, {df_raw.shape[1]} columnas")

# Sanitizar nombres de columnas
df, name_mapping = data.sanitize_columns(df_raw)

# Mostrar mapeo de nombres
if name_mapping:
    print("\n📝 Mapeo de columnas:")
    for orig, safe in name_mapping.items():
        if orig != safe:
            print(f"  '{orig}' → '{safe}'")

In [ ]:
# Librerías estándar
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Manipulación de datos
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

# Agregar mmm_core al path
sys.path.insert(0, str(Path.cwd().parent))
from mmm_core import data

print("✅ Librerías importadas correctamente")